# ETKFでデータ同化するコードのテンプレート

In [1]:
using LinearAlgebra
using HDF5
using Random
using Statistics
using Parameters

In [2]:
@with_kw struct Param{T1,T2}
    #以下，必要なパラメータをかく
     L::T1 = 1.0
    dt::T1 = 0.005
    xmax::T2 = 51
    
    dx::T1 = L/(xmax-1)
    
    nstep::T2 = 900
   

    
    # まず時間発展の計算に必要なパラメータ

    
    # データ同化のパラメータ
    pinit::T2 = 1
    pdis::T2 = 2
    data_true::String = "data_true/data_true"
    data_obs::String = "data_obs/data_obs"
    data_outdir::String = "data_result"

    N::T2 = xmax+1
    p::T2 = length(pinit:pdis:xmax) #観測点の個数
    
    # 変数の合計の個数
    dstep::T2 = 100 # 同化するタイミング
    dstart::T2 = 100 # 同化を開始するステップ
    
    # mはメンバー数，alphaはインフレーションファクタ
    m::T2 = 30
    alpha::T1 = 1.01
    doutput::T2 = 100 # データ同化した結果の出力するタイミング
    
end


Param

In [3]:
# 3 時間発展する部分のコード
function step_kk(para, nu, u, utemp)
    @unpack dx, dt, xmax = para

    c11 = dt/dx/12.0
    c12 = 0.25dt/dx
    c2 = nu*dt/dx^2
    
    for i=3:xmax-2
        utemp[i] = u[i] - c11*u[i]*(-u[i+2] + 8u[i+1] - 8u[i-1] + u[i-2]) -
            c12*abs(u[i])*(u[i+2] - 4u[i+1] + 6u[i] - 4u[i-1] + u[i-2]) +
            c2*(u[i+1]-2u[i]+u[i-1])
    end

    i = 1
    utemp[i] = u[i] - c11*u[i]*(-u[i+2] + 8u[i+1] - 8u[xmax] + u[xmax-1]) -
            c12*abs(u[i])*(u[i+2] - 4u[i+1] + 6u[i] - 4u[xmax] + u[xmax-1]) +
            c2*(u[i+1]-2u[i]+u[xmax])
    i = 2
    utemp[i] = u[i] - c11*u[i]*(-u[i+2] + 8u[i+1] - 8u[i-1] + u[xmax]) -
            c12*abs(u[i])*(u[i+2] - 4u[i+1] + 6u[i] - 4u[i-1] + u[xmax]) +
            c2*(u[i+1]-2u[i]+u[i-1])
    i = xmax
    utemp[i] = u[i] - c11*u[i]*(-u[2] + 8u[1] - 8u[i-1] + u[i-2]) -
            c12*abs(u[i])*(u[2] - 4u[1] + 6u[i] - 4u[i-1] + u[i-2]) +
            c2*(u[1]-2u[i]+u[i-1])
    i= xmax-1
    utemp[i] = u[i] - c11*u[i]*(-u[1] + 8u[i+1] - 8u[i-1] + u[i-2]) -
            c12*abs(u[i])*(u[1] - 4u[i+1] + 6u[i] - 4u[i-1] + u[i-2]) +
            c2*(u[i+1]-2u[i]+u[i-1])
    
    for i=1:xmax
        u[i] = utemp[i]
    end
end
function step(para, nu, u, utemp)

        #step_ud(para, u, utemp)
        step_kk(para, nu, u, utemp)
        
end

step (generic function with 1 method)

In [4]:
# 4 アンサンブルを作るため，異なる初期値を生成する。最初のXaをつくる。
function makeInitialEnsemble(para, nu, Xa)
     @unpack dstart, data_true, dx, dt, m, xmax = para

    #初期値をread
    startfile = data_true*string(dstart+50)*".h5"
    file = h5open(startfile, "r")
    u = read(file, "u")
    close(file)

    rng = MersenneTwister(0)

    for k = 1:m
        for j = 1:xmax
            Xa[j, k] = u[j] + 0.1randn(rng)
        end
        Xa[xmax+1, k] = nu + 0.001*randn()
        
    end
    println("nu_l=",Xa[xmax+1,:])    
    #変数ごとに並べる
end

makeInitialEnsemble (generic function with 1 method)

In [5]:
function makeInitialEnsemble(para, nu, Xa)
    @unpack dstart, data_true, dx, dt, m, xmax = para
    
    for k = 1:m
    # 初期値となるファイル
        startfile = data_true*string(dstart+(k-1)*50)*".h5"
        file = h5open(startfile, "r") 
        u = read(file, "u")
        close(file)
        for j = 1:xmax
            Xa[j, k] = u[j]
        end
        Xa[xmax+1, k] = nu + 0.001*randn()
    end
end

makeInitialEnsemble (generic function with 1 method)

In [6]:
# 5 データ同化に使う変数Xを時間発展方程式の変数に変換する
function X2u(para, u, Xa, k)
    @unpack  xmax = para
    
    for j = 1:xmax
        u[j] = Xa[j, k]
    end
    return Xa[xmax+1, k]
end

X2u (generic function with 1 method)

In [7]:
# 5 時間発展方程式の変数をデータ同化に使う変数Xに変換する
function u2X(para, nu, u, Xa, k)
    @unpack xmax = para

    for j = 1:xmax
        Xa[j, k] = u[j]
    end
    Xa[xmax+1, k] = nu
end

u2X (generic function with 1 method)

In [8]:
# 6 観測値を表すyを作る関数
function makey(para, y, n)
    @unpack dstart, data_obs, p, pinit, pdis, xmax = para

    filename = data_obs*string(n+dstart)*".h5"
    file = h5open(filename, "r")
    uobs = read(file, "u")
    close(file)

    row = 0
    for i = pinit:pdis:xmax
        row = row + 1
        y[row] = uobs[i]
    end
    
end

makey (generic function with 1 method)

In [9]:
# 7 観測行列Hを作る関数
function makeH(para, H)
    @unpack pinit, pdis, p, xmax = para
    
    row = 0
    for i = pinit:pdis:xmax
        row = row + 1
         H[row, i] = 1
    end
    
end

makeH (generic function with 1 method)

In [10]:
# 2(ウ）　観測誤差共分散行列の設定の関数
function makeR(R)
    # の誤差の2乗
    R .= 0.1^2 .* R
end

makeR (generic function with 1 method)

In [11]:
# データ同化した結果の出力
function outputDA(para, )
end


outputDA (generic function with 1 method)

In [12]:
# データ同化を進める関数（一番メインの部分）


function main(para)
    @unpack xmax, p, N, dstep, doutput, dstart, m, nstep, alpha, data_outdir = para

    # 時間発展する計算に必要な変数の宣言を以下にかく
    #ua=
    #va=
    u = zeros(xmax)
    utemp = zeros(xmax)
    utrue = zeros(xmax)

    # データ同化に必要な変数の宣言
    Xa = zeros(Float64, N, m)
    Xf = zeros(Float64, N, m)
    xfm = zeros(Float64, N)
    xam = zeros(Float64, N)
    dXf = zeros(Float64, N, m)
    dXa = zeros(Float64, N, m)
    dY = zeros(Float64, p, m)
    
    R = Matrix{Float64}(I, p, p)
    K = zeros(Float64, N, p)
    H = zeros(p, N)
    y = zeros(p, 1)
    nu = 0.01
    # アンサンブルを作るため，異なる初期値を生成する。最初のXaをつくる。
    makeInitialEnsemble(para, nu, Xa)

    #　y, H, Rの設定
    makeH(para, H)
    makeR(R)

    rms_a = zeros(Float64, nstep)
    rms_o = zeros(Float64, nstep)
    step_nu = zeros(Float64, nstep)

    # 時間発展。時間発展方程式を解きながら，データ同化もする。
    for n=1:nstep
        # 新しいステップの予測値Xfの計算
        for l=1:m
            nu = X2u(para, u, Xa, l) # 5
            #println("nu=",nu)
            step(para, nu, u, utemp) #時間発展
            u2X(para, nu, u, Xf, l)　# 5
        end
        
        xfm = mean(Xf, dims=2)
        for l=1:m
            dXf[:,l] = Xf[:,l] - xfm[:]
        end
        
        if mod(n, dstep) == 0
            #inflation 
            dXf = alpha * dXf
            dY = H*dXf
            K = dXf*dY'*inv(dY*dY' + (m-1)*R)
            #　平均の解析値xamの計算
            xam = xfm + K*(y - H*xfm)

            makey(para, y, n)
            # 固有値の計算
            val, vec = eigen( Symmetric(I - dY'*inv(dY*dY' + (m-1)*R)*dY) )
            #println(val)
            T = vec*sqrt( Diagonal(val) ) * vec'
            #println("T=", size(T))
            
            dXa = dXf*T
            
            for l = 1:m
                Xa[:,l] = xam[:] + dXa[:,l]
            end

        # データ同化しないステップはXfをXaにコピーするだけ。
        else
            xam[:] = xfm[:]
            Xa[:,:] = Xf[:,:]
        end
        #Xa[:,i+1] = Xf[:,i+1]
        #Pa[:,:, i+1] = Pa[:,:, i]

        if n%5 == 0
            filename = "data_true/data_true"*string(n+dstart)*".h5"
            file = h5open(filename, "r")
            utrue .= read(file, "u")
            close(file)
            makey(para, y, n)
        end
        
        rms_a[n] = norm(xam[1:N-1]-utrue[:])/sqrt(N-1)
        rms_o[n] = norm(H[:, 1:N-1]*utrue[:]-y)/sqrt(p)
        step_nu[n] = xam[xmax+1]

        # データ同化した結果の出力
        if mod(n, doutput) == 0
        #  outputDA(xam)
            u .= xam[1:xmax]
            nu = xam[xmax+1]
            h5open(data_outdir*"/data_result"*string(n+dstart)*".h5", "w") do file
                write(file,"nu",nu)
                write(file, "u",u)
                write(file, "u_xa1", Xa[1:xmax,1])
                write(file, "u_xa2", Xa[1:xmax,2])
            end
            println("nu_da=",nu)
        end
    end

    h5open(data_outdir*"/data_RMS.h5", "w") do file
        write(file,"rms_a",rms_a)
        write(file,"rms_o",rms_o)
        write(file,"step_nu",step_nu)
    end
    
    println("rms_a=",rms_a)
    println("rms_o=",rms_o)
    
end

main (generic function with 1 method)

In [13]:
# 計算実行
para = Param(alpha=1.05, dstep=5, pdis=3, doutput=10, dstart=10)



Param{Float64, Int64}
  L: Float64 1.0
  dt: Float64 0.005
  xmax: Int64 51
  dx: Float64 0.02
  nstep: Int64 900
  pinit: Int64 1
  pdis: Int64 3
  data_true: String "data_true/data_true"
  data_obs: String "data_obs/data_obs"
  data_outdir: String "data_result"
  N: Int64 52
  p: Int64 17
  dstep: Int64 5
  dstart: Int64 10
  m: Int64 30
  alpha: Float64 1.05
  doutput: Int64 10


In [14]:
@time main(para)

nu_da=0.01010159797633926
nu_da=0.009943759030261269
nu_da=0.009150991261240032
nu_da=0.008244515797310944
nu_da=0.00619756138809422
nu_da=0.005350026278162911
nu_da=0.005171677841530762
nu_da=0.0037547575987535156
nu_da=0.0037172942645812878
nu_da=0.003821738689178147
nu_da=0.003504224307000362
nu_da=0.003318214661575294
nu_da=0.0029115834439508276
nu_da=0.0030033293198579052
nu_da=0.0029077852870623036
nu_da=0.0031605304610949855
nu_da=0.0026181133156539162
nu_da=0.0029631155031759893
nu_da=0.0025860480722323218
nu_da=0.002175324444946734
nu_da=0.002190770122305132
nu_da=0.0018011563898317098
nu_da=0.0022601513880608176
nu_da=0.0020677610656527155
nu_da=0.0021973218054146103
nu_da=0.001899341193433102
nu_da=0.002035469234708826
nu_da=0.0024098376543577435
nu_da=0.00268649880792399
nu_da=0.0026141484124715835
nu_da=0.002816526223495543
nu_da=0.0028791538160940914
nu_da=0.003246447396687385
nu_da=0.002755357272601199
nu_da=0.0020394651084860204
nu_da=0.00246588170489367
nu_da=0.0025040